In [99]:
import pandas as pd
from nltk.corpus import stopwords
import tweepy
import json
import numpy as np
import datetime as dt
import requests

In [100]:
# open file with keys and set the path to your credentials JSON file
# you'll need to replace my file with yours
credentials = "twitter_keys.json"
with open(credentials, "r") as keys:
    api_tokens = json.load(keys)

In [101]:
bearer_token = api_tokens["bearer_token"]

In [102]:
# we only want status info so we won't need the higher level access
client = tweepy.Client(bearer_token=bearer_token)

In [103]:
client

In [104]:
duolingo_data = client.search_recent_tweets(
    # you only need spaces between your filters, so we are getting searches for #Duolingo365 while ignoring retweets and only getting tweets in english
    query = "#Duolingo365 -is:retweet lang:en",  
    # only show 100 results
    max_results = 100
)

# print out tweets
duolingo_data.data

[<Tweet id=1600467036312047616 text='I’m a Galactic Legend! What’s your Duolingo learner style? #Duolingo365 https://t.co/Pc0RzojTan'>,
 <Tweet id=1600466945450463232 text='Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/INlveNXKBw'>,
 <Tweet id=1600466927201447936 text='Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/cWlBlr2FHi'>,
 <Tweet id=1600466896209453056 text='Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/CeNZnJntIy'>,
 <Tweet id=1600466812734689280 text='Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/BekRhZkhII'>,
 <Tweet id=1600466766815117312 text='Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/aJwcRh6hqe'>,
 <Tweet id=1600466757361496064 text='Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/MyqVXnTr35'>,
 <Tweet id=1600466740650987520 text=

In [105]:
type(duolingo_data.data)

list

In [106]:
duolingo_tweets = []
for tweet in duolingo_data.data:
    # .text gives just the text of the tweets and ignores the meta data ie. tweet id's
    duolingo_tweets.append(tweet.text)
    
duolingo_tweets

['I’m a Galactic Legend! What’s your Duolingo learner style? #Duolingo365 https://t.co/Pc0RzojTan',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/INlveNXKBw',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/cWlBlr2FHi',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/CeNZnJntIy',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/BekRhZkhII',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/aJwcRh6hqe',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/MyqVXnTr35',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/DAPjUqT88r',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://t.co/oEaYTA5HlC',
 'Look how much I learned on Duolingo in 2022! How did you do? #Duolingo365 https://

In [107]:
piers_data = client.search_recent_tweets(
    query = "Piers Morgan -is:retweet lang:en",  # searches for #WPP while ignoring retweets
    max_results = 100
)

piers_data.data


[<Tweet id=1600467042393460736 text='Erik said the same things, few weeks later he rant to Piers Morgan. https://t.co/PDacby1UBY'>,
 <Tweet id=1600466702688321536 text='@talkSPORT Piers Morgan is coming for you Fernando 😱 when is the next interview, will he do the interview now or wait until he goes back to his clu... oh wait 🤔'>,
 <Tweet id=1600466515521839105 text='Piers Morgan needs to stop thinking eating meat in front of a vegan on telly is a moment'>,
 <Tweet id=1600466386337558531 text='@Doc_Gwarzo @freyizit He needs to run out of people to report to Piers Morgan first.'>,
 <Tweet id=1600466307350409218 text="The moment Ronaldo turns his back on Piers Morgan is the day Piers Morgan's unhinged obsession starts.\nA good example is Meghan.">,
 <Tweet id=1600466261661523969 text='listening to this kanye interview with piers morgan has me all over the place this man has intelligent valid thoughts but does not know how to elaborate himself the correct way !!!'>,
 <Tweet id=16004661889

In [108]:
piers_tweets = []
for tweet in piers_data.data:
    piers_tweets.append(tweet.text)
    
piers_tweets

['Erik said the same things, few weeks later he rant to Piers Morgan. https://t.co/PDacby1UBY',
 '@talkSPORT Piers Morgan is coming for you Fernando 😱 when is the next interview, will he do the interview now or wait until he goes back to his clu... oh wait 🤔',
 'Piers Morgan needs to stop thinking eating meat in front of a vegan on telly is a moment',
 '@Doc_Gwarzo @freyizit He needs to run out of people to report to Piers Morgan first.',
 "The moment Ronaldo turns his back on Piers Morgan is the day Piers Morgan's unhinged obsession starts.\nA good example is Meghan.",
 'listening to this kanye interview with piers morgan has me all over the place this man has intelligent valid thoughts but does not know how to elaborate himself the correct way !!!',
 '@FabrizioRomano He’s going to tell Piers Morgan 😭 https://t.co/U24n7AKkDU',
 "Piers Morgan wades into Harry and Meghan fakery row after claims over 'intrusive' photo https://t.co/HrQaAizsbW",
 '@ladbible News just in, Piers Morgan acts 

In [121]:
import random

# make a function called get_features with the parameter text
def setup_features(text):
    # create array of keywords
    # make all words lower case because all stopwords are lowercase
    text = text.lower()
    # split up the tweets into an array of words
    words_list = text.split()
    # get rid of english stop words like if, or, the, you may wanna include stopwords in other languages depending on the tweets you are getting
    stop_words = stopwords.words('english')
    
    # it needs to match exactly
        # characters_i_hate = ['@', '-', 'https', '.']
        # stop_words.extend(characters_i_hate)
    
    keywords = [ word for word in words_list if word not in stop_words and 'http' not in word]
    random.shuffle(keywords)
    # only get first 3 keywords, if it's more than 3 keywords ignore them.
    while len(keywords) < 3:
        keywords.append('None')
        
    return {'first_keyword': keywords[0], 'second_keyword': keywords[1], 'third_keyword': keywords[2]}

In [122]:
# create a list of tuples containing both the featureset and the lable
# features and labels list will contain my features and my labels
features_and_labels = []
for tweet in duolingo_tweets:
    features_and_labels.append((setup_features(tweet), 'Duolingo'))

In [123]:
for tweet in piers_tweets:
    features_and_labels.append((setup_features(tweet), 'Piers Morgan'))

In [124]:
features_and_labels

[({'first_keyword': 'style?',
   'second_keyword': '#duolingo365',
   'third_keyword': 'duolingo'},
  'Duolingo'),
 ({'first_keyword': 'look',
   'second_keyword': 'much',
   'third_keyword': '2022!'},
  'Duolingo'),
 ({'first_keyword': '2022!',
   'second_keyword': '#duolingo365',
   'third_keyword': 'much'},
  'Duolingo'),
 ({'first_keyword': '2022!',
   'second_keyword': 'duolingo',
   'third_keyword': 'much'},
  'Duolingo'),
 ({'first_keyword': 'do?',
   'second_keyword': 'learned',
   'third_keyword': 'look'},
  'Duolingo'),
 ({'first_keyword': 'learned',
   'second_keyword': 'much',
   'third_keyword': '#duolingo365'},
  'Duolingo'),
 ({'first_keyword': 'do?',
   'second_keyword': '#duolingo365',
   'third_keyword': '2022!'},
  'Duolingo'),
 ({'first_keyword': 'do?',
   'second_keyword': '2022!',
   'third_keyword': 'duolingo'},
  'Duolingo'),
 ({'first_keyword': '2022!',
   'second_keyword': 'much',
   'third_keyword': 'look'},
  'Duolingo'),
 ({'first_keyword': '#duolingo365',


In [125]:
# ALWAYS SHUFFLE YOUR FEATURES AND LABLES
# if it is in order and you take the top 80% the entire testing data will just be piers morgan
random.shuffle(features_and_labels)

In [126]:
features_and_labels

[({'first_keyword': 'idiot',
   'second_keyword': 'sooo',
   'third_keyword': 'piers'},
  'Piers Morgan'),
 ({'first_keyword': '@mkpo_manny',
   'second_keyword': 'tweet',
   'third_keyword': 'tire'},
  'Piers Morgan'),
 ({'first_keyword': 'piers', 'second_keyword': 'hag', 'third_keyword': 'ten'},
  'Piers Morgan'),
 ({'first_keyword': 'do?',
   'second_keyword': 'duolingo',
   'third_keyword': 'learned'},
  'Duolingo'),
 ({'first_keyword': 'pearl',
   'second_keyword': 'faruzan',
   'third_keyword': 'adel'},
  'Piers Morgan'),
 ({'first_keyword': '@piersmorgan',
   'second_keyword': 'seen',
   'third_keyword': 'morgan'},
  'Piers Morgan'),
 ({'first_keyword': 'i’m',
   'second_keyword': 'duolingo',
   'third_keyword': 'learner!'},
  'Duolingo'),
 ({'first_keyword': 'harassment',
   'second_keyword': 'protect',
   'third_keyword': 'harry'},
  'Piers Morgan'),
 ({'first_keyword': 'much',
   'second_keyword': 'look',
   'third_keyword': '#duolingo365'},
  'Duolingo'),
 ({'first_keyword':

In [127]:
# split 80:20 training data and testing data

import math

# get how many elements in my list is 80%
split_num = math.floor(len(features_and_labels) * .8) # .floor = round down 

split_num

156

In [128]:
# training set = 80% (split_num) of features_and_lables 
training_set = features_and_labels[:split_num]
# testing_set = 20% of features_and_lables
testing_set = features_and_labels[split_num:]

In [129]:
import nltk

# create our classifier

# My baseline is 50% because I have two labels/outcomes, if I had 4 I would have a baseline of 25%
my_classifier = nltk.NaiveBayesClassifier.train(training_set)

In [130]:
my_classifier.classify(setup_features("Big Piers Morgan getting told off by specsavers has made my day."))

'Piers Morgan'

In [131]:
# test our accuracy: if u can get it over 50% (bc we only have 2 labels) that's sick
nltk.classify.accuracy(my_classifier, testing_set)

0.95

In [133]:
classifier.show_most_informative_features(10)

Most Informative Features
           first_keyword = 'look'         Duolin : Piers  =      8.7 : 1.0
          second_keyword = 'look'         Duolin : Piers  =      8.7 : 1.0
           third_keyword = 'look'         Duolin : Piers  =      8.7 : 1.0
